In [51]:
import os
from datetime import datetime, timedelta
import pandas as pd
from pathlib import Path
from lib import Lab

# Initialize lab
lab = Lab('mev-relays', '../config.yaml')
lab.setup()
lab.setup_pandaops_clickhouse()
log = lab.log

# Get notebook specific config
notebook_config = lab.get_notebook_config()

writer = lab.get_data_writer()

pandaops_clickhouse_client = lab.get_pandaops_clickhouse_client()

log.info("Good to go!")

# EARLY EXIT. We Don't use this notebook yet.
return


2025-01-10 15:00:18,874 - mev-relays - INFO - Good to go!


: 

In [2]:
mev_relays_config = lab.get_notebook_config().as_mev_relays()
mev_relays_config

MevRelays(time_windows=[TimeWindow(file='last_30_days', step='1d', label='Last 30d', range='-30d'), TimeWindow(file='last_1_day', step='1h', label='Last 1d', range='-1d'), TimeWindow(file='last_6h', step='5m', label='Last 6h', range='-6h')], networks=['mainnet'])

In [4]:
validator_query = text("""
    SELECT 
        `index` as validator_index,
        entity
    FROM ethseer_validator_entity
    WHERE 
        meta_network_name = :network
""")
validator_entities = pd.DataFrame(
    pandaops_clickhouse_client.execute(validator_query, {"network": mev_relays_config.networks[0]}).fetchall(),
    columns=['validator_index', 'entity']
)
validator_entities

,validator_index,entity
0,88035,whale_0x3230
1,985037,whale_0x9f1d
2,985038,whale_0x9f1d
3,985039,whale_0x9f1d
4,985040,whale_0x9f1d
...,...,...
1535620,983558,rocketpool
1535621,983572,solo_stakers
1535622,983573,solo_stakers
1535623,983671,solo_stakers


In [5]:
from sqlalchemy import text
from datetime import datetime, timezone

query = text("""
    SELECT
        validator_index,
        slot,
        gas_limit,
        timestamp,
        relay_name
    FROM
        mev_relay_validator_registration
    WHERE
        event_date_time >= :start_date
        AND event_date_time < :end_date
        AND meta_network_name = :network
    GROUP BY
        validator_index,
        slot,
        gas_limit,
        timestamp,
        relay_name
    ORDER BY
        slot,
        validator_index,
        relay_name
""")

# Get last 2.5 weeks window
end_date = datetime.now(timezone.utc)
start_date = end_date - timedelta(days=16)

# Format dates without microseconds for Clickhouse
start_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
end_str = end_date.strftime('%Y-%m-%d %H:%M:%S')

log.info("Fetching relay registrations for last 16 days")

registrations = pd.read_sql(
    query, 
    pandaops_clickhouse_client, 
    params={
        "start_date": start_str,
        "end_date": end_str,
        "network": mev_relays_config.networks[0]
    }
)

# Add proposer entity to registrations
registrations = registrations.merge(validator_entities, on='validator_index', how='left')
registrations


2025-01-10 14:09:45,607 - mev-relays - INFO - Fetching relay registrations for last 2.5 weeks


,validator_index,slot,gas_limit,timestamp,relay_name,entity
0,494481,0,30000000,1606824023,Aestus,solo_stakers
1,494481,0,30000000,1606824023,Agnostic Gnosis,solo_stakers
2,494481,0,30000000,1606824023,Titan Relay,solo_stakers
3,494481,0,30000000,1606824023,Ultra Sound,solo_stakers
4,556545,0,30000000,1606824023,Aestus,solo_stakers
...,...,...,...,...,...,...
11929970,781433,10804781,36000000,1736481404,Titan Relay,coinbase
11929971,781628,10804781,36000000,1736481404,BloXroute Max Profit,coinbase
11929972,781628,10804781,36000000,1736481404,Titan Relay,coinbase
11929973,782602,10804781,36000000,1736481404,BloXroute Max Profit,coinbase


In [14]:
# Get latest epoch's validators
query = text("""
    SELECT 
        index,
        status
    FROM canonical_beacon_validators
    WHERE 
        epoch = (
            SELECT MAX(epoch) 
            FROM canonical_beacon_validators 
            WHERE meta_network_name = :network
            AND epoch_start_date_time >= NOW() - INTERVAL 14 DAY
        )
        AND meta_network_name = :network
        AND epoch_start_date_time >= NOW() - INTERVAL 14 DAY
""")

validators = pd.read_sql(
    query,
    pandaops_clickhouse_client,
    params={"network": mev_relays_config.networks[0]}
)
validators


,index,status
0,2,active_ongoing
1,3,active_ongoing
2,4,active_ongoing
3,8,active_ongoing
4,11,active_ongoing
...,...,...
1759949,893333,active_ongoing
1759950,893335,active_ongoing
1759951,893343,active_ongoing
1759952,893344,active_ongoing


In [23]:
non_active_validators = validators[validators['status'] != 'active_ongoing']
non_active_validators

active_validators = validators[validators['status'] == 'active_ongoing']
active_validators

,index,status
0,2,active_ongoing
1,3,active_ongoing
2,4,active_ongoing
3,8,active_ongoing
4,11,active_ongoing
...,...,...
1759949,893333,active_ongoing
1759950,893335,active_ongoing
1759951,893343,active_ongoing
1759952,893344,active_ongoing


In [29]:
active_registrations = registrations[~registrations['validator_index'].isin(non_active_validators['index'])]
active_registrations['date'] = pd.to_datetime(active_registrations['timestamp'], unit='s').dt.date
active_registrations


/var/folders/fq/bp58gxy12nd2b32pw1y8wy040000gn/T/ipykernel_28284/3333959086.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_registrations['date'] = pd.to_datetime(active_registrations['timestamp'], unit='s').dt.date


,validator_index,slot,gas_limit,timestamp,relay_name,entity,date
0,494481,0,30000000,1606824023,Aestus,solo_stakers,2020-12-01
1,494481,0,30000000,1606824023,Agnostic Gnosis,solo_stakers,2020-12-01
2,494481,0,30000000,1606824023,Titan Relay,solo_stakers,2020-12-01
3,494481,0,30000000,1606824023,Ultra Sound,solo_stakers,2020-12-01
4,556545,0,30000000,1606824023,Aestus,solo_stakers,2020-12-01
...,...,...,...,...,...,...,...
11929970,781433,10804781,36000000,1736481404,Titan Relay,coinbase,2025-01-10
11929971,781628,10804781,36000000,1736481404,BloXroute Max Profit,coinbase,2025-01-10
11929972,781628,10804781,36000000,1736481404,Titan Relay,coinbase,2025-01-10
11929973,782602,10804781,36000000,1736481404,BloXroute Max Profit,coinbase,2025-01-10


In [18]:
# Ensure we've got the most recent registrations with a unique key of validator index and relay name
most_recent_registrations_per_relay = active_registrations.groupby(['validator_index', 'relay_name']).last().reset_index()
most_recent_registrations_per_relay


,validator_index,relay_name,slot,gas_limit,timestamp,entity
0,0,Aestus,8708041,30000000,1711320515,solo_stakers
1,0,Agnostic Gnosis,8708041,30000000,1711320515,solo_stakers
2,0,BloXroute Max Profit,10681573,30000000,1735002899,solo_stakers
3,0,Eden Network,10681573,30000000,1735002899,solo_stakers
4,0,Flashbots,8708041,30000000,1711320515,solo_stakers
...,...,...,...,...,...,...
6708979,1734312,BloXroute Regulated,10799168,30000000,1736414039,None
6708980,1734312,Flashbots,10796057,30000000,1736376707,None
6708981,1734312,Manifold,10796057,30000000,1736376707,None
6708982,1734312,Titan Relay,10799168,30000000,1736414039,None


In [50]:
# Ensure we've got the most recent registrations with a unique key of validator index ordered by timestamp
# Filter for active validators only
most_recent_registrations = active_registrations[active_registrations['validator_index'].isin(active_validators['index'])] \
    .groupby(['validator_index']).last().reset_index()
most_recent_registrations


,validator_index,slot,gas_limit,timestamp,relay_name,entity,date
0,0,10681573,30000000,1735002899,Titan Relay,solo_stakers,2024-12-24
1,1,10681573,30000000,1735002899,Titan Relay,solo_stakers,2024-12-24
2,2,10681573,30000000,1735002899,Titan Relay,solo_stakers,2024-12-24
3,3,10681573,30000000,1735002899,Titan Relay,solo_stakers,2024-12-24
4,4,10681573,30000000,1735002899,Titan Relay,solo_stakers,2024-12-24
...,...,...,...,...,...,...,...
1036498,1734308,10799168,30000000,1736414039,Titan Relay,None,2025-01-09
1036499,1734309,10799168,30000000,1736414039,Titan Relay,None,2025-01-09
1036500,1734310,10799168,30000000,1736414039,Titan Relay,None,2025-01-09
1036501,1734311,10799168,30000000,1736414039,Titan Relay,None,2025-01-09


In [48]:
# Start building out the data we want to write to file

total_validators_registered = len(most_recent_registrations)
total_active_validators = len(active_validators)

## Count the number of validators registered to each relay
relay_totals = most_recent_registrations.groupby('relay_name').size().reset_index(name='count')
relay_totals




,relay_name,count
0,Aestus,1994
1,Agnostic Gnosis,12105
2,BloXroute Max Profit,43775
3,BloXroute Regulated,43515
4,Eden Network,16
5,Flashbots,12401
6,Manifold,4673
7,Titan Relay,735983
8,Ultra Sound,175433
9,Wenmerge,6608


In [39]:
# Get the number of validators who are not registered to any relay and their entities
non_registered_validators = active_validators[~active_validators['index'].isin(most_recent_registrations['validator_index'])]
non_registered_validators = non_registered_validators.merge(validator_entities, left_on='index', right_on='validator_index')

# Count per entity, show the top 10
non_registered_validators.groupby('entity').size().sort_values(ascending=False).head(10)


entity
solo_stakers     7756
whale_0x24d6     1014
whale_0xe43c     1011
abyss_finance     616
whale_0xefa0      507
whale_0x39fd      329
whale_0x900c      309
whale_0x1d5b      308
whale_0x10d5      304
whale_0xe3f7      290
dtype: int64

In [49]:
# Write summary data to JSON
summary = {
    "total_active_validators": int(total_active_validators),
    "total_validators_registered": int(total_validators_registered),
    "relay_registrations": relay_totals.to_dict('records'),
    "top_unregistered_entities": non_registered_validators.groupby('entity').size().sort_values(ascending=False).head(10).to_dict()
}


lab.write_json('mev-relays-summary.json', summary)




{'total_active_validators': 1070507,
 'total_validators_registered': 1036503,
 'relay_registrations': [{'relay_name': 'Aestus', 'count': 1994},
  {'relay_name': 'Agnostic Gnosis', 'count': 12105},
  {'relay_name': 'BloXroute Max Profit', 'count': 43775},
  {'relay_name': 'BloXroute Regulated', 'count': 43515},
  {'relay_name': 'Eden Network', 'count': 16},
  {'relay_name': 'Flashbots', 'count': 12401},
  {'relay_name': 'Manifold', 'count': 4673},
  {'relay_name': 'Titan Relay', 'count': 735983},
  {'relay_name': 'Ultra Sound', 'count': 175433},
  {'relay_name': 'Wenmerge', 'count': 6608}],
 'top_unregistered_entities': {'solo_stakers': 7756,
  'whale_0x24d6': 1014,
  'whale_0xe43c': 1011,
  'abyss_finance': 616,
  'whale_0xefa0': 507,
  'whale_0x39fd': 329,
  'whale_0x900c': 309,
  'whale_0x1d5b': 308,
  'whale_0x10d5': 304,
  'whale_0xe3f7': 290}}